# MNIST classifier from nnBuilder

This is a simple implementation of a MNIST classifier using the nnBuilder framework. The network is essentially identical to the one from the TensorFlow tutorial at https://www.tensorflow.org/tutorials/mnist/pros/.

In [1]:
import numpy as np
import tensorflow as tf
import sys
import matplotlib.pyplot as plt
sys.path.append('../nnBuilder')
from nnLayer import *
from nnInput import *
from nnTrainer import *
from nnHandler import *

In [ ]:
test_split_n=10 #Avoid OOM
data=Layer(type="MNIST",batch=128)
data_test=Layer(type="MNIST",test=True)
data_test=Layer(x=data_test,type="Batch_Slice",batch=10000//test_split_n)
layers=[]
layers.append(dict(type="Convolution",pad="SAME",window=5,stride=1,size=32*4,relu=True))
layers.append(dict(type="Pool",pad="SAME",window=2,stride=2,pool_type="max"))
layers.append(dict(type="Convolution",pad="SAME",window=5,stride=1,size=64*4,relu=True))
layers.append(dict(type="Pool",pad="SAME",window=2,stride=2,pool_type="max"))
layers.append(dict(type="Relu",size=64*8))
layers.append(dict(type="Linear",size=10,in_features=["Dropout"]))
network_def=dict(type="Network",layers=layers)
network=Layer(x=data,**network_def)
network_test=network.copy(x=data_test,share_vars=True,test=True)
trainer=ClassifierTrainer(network=network,optimizer="adam")
tester=ClassifierTrainer(network=network_test,test=True)
sess=SessManager(data,network,trainer,data_test,network_test,tester)
sess.start()

Extracting /tmp/tensorflow/mnist/input_data\train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-labels-idx1-ubyte.gz
train
Extracting /tmp/tensorflow/mnist/input_data\train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-labels-idx1-ubyte.gz
test
Starting new session


In [ ]:
batches_per_step=100
trainer.train(0,keep_rate=1.)
trains=[trainer.eval_error()[1]]
tests=[tester.eval_error()[1]]
for i in range(100):
    trainer.train(batches_per_step,keep_rate=0.5,l2reg=1e-2)
    trainer.train(0,keep_rate=1.)#To eval without dropout, fix needed
    trains.append(trainer.eval_error(n=100)[1]) #Training error, reaches about 0.02% with more steps
    tests.append(tester.eval_error(n=test_split_n)[1]) #Testing error, reaches about 0.7% with more steps

Error rate (train): 0.9
Error rate (test): 0.897
Error rate (train): 0.0666406
Error rate (test): 0.0585
Error rate (train): 0.0430469
Error rate (test): 0.0407
Error rate (train): 0.0321875
Error rate (test): 0.0294
Error rate (train): 0.0263281
Error rate (test): 0.0232
Error rate (train): 0.0200781
Error rate (test): 0.0207
Error rate (train): 0.0175781
Error rate (test): 0.0168
Error rate (train): 0.0170313
Error rate (test): 0.0165
Error rate (train): 0.0150781
Error rate (test): 0.0166
Error rate (train): 0.0126563
Error rate (test): 0.0157
Error rate (train): 0.0151563
Error rate (test): 0.0154
Error rate (train): 0.0117188
Error rate (test): 0.0138
Error rate (train): 0.0135156
Error rate (test): 0.0152
Error rate (train): 0.0136719
Error rate (test): 0.0139
Error rate (train): 0.01125
Error rate (test): 0.0122
Error rate (train): 0.00921875
Error rate (test): 0.0125
Error rate (train): 0.00898437
Error rate (test): 0.0105
Error rate (train): 0.0121094
Error rate (test): 0.013


In [ ]:
#Plot the training and testing error rates (add more steps to see overfitting)
x=[batches_per_step*i for i in range(len(trains))]
plt.plot(x,trains, label="training")
plt.plot(x,tests, label="test")
plt.legend()
plt.ylim(0, .05)
plt.show()

In [ ]:
network.save() #The full network definition
save=network.save() #The full network definition
assert(Layer(x=data,**save).save()==save) #Consistency
print(save==network_def) #Saving adds stuff, should still be equivalent
save

In [ ]:
import sys, os, seaborn, time
t0=time.time()
trainer.train(600000//128,keep_rate=0.5)
time.time()-t0